<a href="https://colab.research.google.com/github/Filocava99/Autonomous-Driving-Classification/blob/main/AutonomousDrivingClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
!pip install Augmentor

import os
import glob
import numpy as np
import random
from PIL import Image
from matplotlib import pyplot as plt
import Augmentor
import shutil
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, losses
from tensorflow import keras as K
from google.colab.patches import cv2_imshow

## Download training set

In [ ]:
!wget http://bias.csr.unibo.it/VR/Challenge/AutonomousDriving/train.zip
!unzip /content/train.zip
!rm /content/train.zip

## Data augmentation

In [ ]:
def augmentPath(path, label):
  basePath = path + "/" + label
  if os.path.exists(basePath + "/output"):
    print("Deleting " + basePath + "/output")
    shutil.rmtree(basePath + "/output")
  p1 = Augmentor.Pipeline(basePath, save_format="PNG")
  p1.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
  p1.skew_tilt(0.4, 0.35)
  p1.random_distortion(0.05, 64, 64, 1)
  p1.flip_random(0.4)
  p1.shear(0.2, max_shear_left=5, max_shear_right=5)
  p1.sample(5000)
  shutil.move(basePath + "/output", "/content/train_new/" + label)
if os.path.exists("/content/train_new"):
  shutil.rmtree("/content/train_new")
for dir in os.listdir("/content/train"):
  augmentPath("/content/train", dir)

## Donwload evaluation set

In [ ]:
!wget http://bias.csr.unibo.it/VR/Challenge/AutonomousDriving/eval.zip
!unzip /content/eval.zip
!rm /content/eval.zip

## Utilities

In [ ]:
def load_data(data_folder_path):
    image_list=[]
    label_list=[]
    class_folder_path_list=glob.glob(data_folder_path+'/*/')
    for class_folder_path in class_folder_path_list:
        class_folder_path=class_folder_path.replace('\\','/')
        folder_label=os.path.basename(class_folder_path[:-1])

        for image_file_path in glob.glob(class_folder_path+'/*.PNG'):
            image_file_path=image_file_path.replace('\\','/')
            image=Image.open(image_file_path)
            image_list.append(np.asarray(image)[:,:,:3])
            label_list.append(int(folder_label))
    return np.asarray(image_list),np.asarray(label_list)

## Loads training set

In [ ]:
db_path = '/content/train_new/'
class_names=['Pedestrian','Cyclist','Car','Truck','Tram','Tricycle']

X,y = load_data(db_path)

from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.25)

print('Shape training set: {}'.format(train_x.shape))
print('Shape validation set: {}'.format(val_x.shape))

## Images visualization

In [ ]:
_, axs = plt.subplots(1, len(class_names),figsize=(20, 10))
for i in range(len(class_names)):
  train_class_pos=np.where(train_y==i)
  rnd_idx=random.randint(0,len(train_class_pos[0]))
  p=train_class_pos[0][rnd_idx]
  axs[i].imshow(train_x[p]),axs[i].axis('off'),axs[i].set_title(class_names[train_y[p]])

## Loads evaluation set

In [ ]:
def loadImages(path):
  image_list=[]
  image_names=[]
  for filename in os.listdir(path):
    image_names.append(filename)
    image=Image.open(path+"/"+filename)
    image_list.append(np.asarray(image)[:,:,:3])
  return np.asarray(image_list), np.asarray(image_names)

db_path = '/content/eval/'
test_x, test_names = loadImages(db_path)
print(test_names[0])
print('Shape training set: {}'.format(test_x.shape))

## Evaluation set images visualization

In [ ]:
_, axs = plt.subplots(1, 6,figsize=(20, 10))
for i in range(6):
  axs[i].imshow(test_x[i]),axs[i].axis('off'),axs[i].set_title(test_names[i])

## ResNet50 import
The fully connected layer at the end is removed. All the other layers will be freezed to avoid being trained again, except for the last twenty layers.

Imports ResNet50.

In [ ]:
from tensorflow import keras
input_t = keras.Input(shape=(224,224,3))
res_model = keras.applications.ResNet50(include_top=False, weights="imagenet", input_tensor=input_t)

Freezes the layers.

In [ ]:
for layer in res_model.layers[:143]:
  layer.trainable = False

Verifies that the layers are correctly freezed.

In [ ]:
for i, layer in enumerate(res_model.layers):
  print(i, layer.name, "-", layer.trainable)

Adds new layers to the model and then starts the training.

In [ ]:
model = K.models.Sequential()
model.add(K.layers.Lambda(lambda x: tf.image.resize(x, (224, 224))))
model.add(res_model)
model.add(K.layers.Flatten())
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(256, activation='relu'))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(128, activation='relu'))
model.add(K.layers.Dropout(0.3))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(64, activation='relu'))
model.add(K.layers.Dropout(0.3))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(6, activation='softmax'))
model.compile(loss=losses.sparse_categorical_crossentropy,
              optimizer=K.optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])

checkpointer = K.callbacks.ModelCheckpoint(filepath='trained-model2.h5',
                                           monitor="val_accuracy",
                                           verbose=1, save_best_only=True)
history = model.fit(train_x, train_y, validation_data=(val_x, val_y), batch_size=64, epochs=10, verbose=1,
                    shuffle=True,
                    callbacks=[checkpointer])
model.summary()

Visualizes convergence trend (accuracy of training and evaluation sets).

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(15,15))
axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Loss')
axs[0].legend(['Train','Val'])
axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Accuracy')
axs[1].legend(['Train', 'Val'])

## Predictions on the test set

In [ ]:
result = model.predict(test_x)